<a href="https://colab.research.google.com/github/Raph33AI/SonicSense-AI/blob/main/SonicSense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:

import requests
from bs4 import BeautifulSoup

url = "https://charts.spotify.com/viral/global/weekly/latest"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

trending_titles = []
for track_row in soup.select("table tbody tr"):
    title_element = track_row.find("td", class_="chart-table-track").find("strong")
    if title_element:
        title = title_element.text.strip()
        trending_titles.append(title)

print("The 100 most trending titles on Spotify this week:")
for i, title in enumerate(trending_titles):
    print(f"{i+1}. {title}")


The 100 most trending titles on Spotify this week:


In [24]:

import requests
import base64
import random

# Vos identifiants d'application
CLIENT_ID = "dffbe0f471154691b105df32348a2378"
CLIENT_SECRET = "cd114e5ad1654451a84a1d3b9e32c7ee"

# Fonction pour obtenir l'access token Spotify
def get_access_token():
    auth_string = f"{CLIENT_ID}:{CLIENT_SECRET}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    token_url = "https://accounts.spotify.com/api/token"
    token_data = {
        "grant_type": "client_credentials"
    }
    token_headers = {
        "Authorization": f"Basic {auth_base64}"
    }

    token_response = requests.post(token_url, data=token_data, headers=token_headers)

    if token_response.status_code == 200:
        return token_response.json()["access_token"]
    else:
        print(f"Erreur lors de la récupération du token: {token_response.status_code}")
        return None

# Obtenir l'access token
ACCESS_TOKEN = get_access_token()

if ACCESS_TOKEN:
    headers = {
        "Authorization": f"Bearer {ACCESS_TOKEN}"
    }

    genre_demandé = input("Entrez un genre musical : ")

    # Définir un offset aléatoire (par exemple, entre 0 et 100)
    offset_aleatoire = random.randint(0, 100)

    search_url = "https://api.spotify.com/v1/search"
    search_params = {
        "q": f"genre:{genre_demandé}",
        "type": "track",
        "limit": 5,  # Limite à 5 résultats
        "offset": offset_aleatoire  # Ajoute l'offset aléatoire
    }
    search_response = requests.get(search_url, headers=headers, params=search_params)

    if search_response.status_code == 200:
        search_data = search_response.json()
        tracks = search_data["tracks"]["items"]

        print("Artiste - Album - Date - Tempo - Danceability - Energy - Valence - Lien")
        print("---------------------------------------------------------------------")

        for track in tracks:
            artists = ", ".join([artist["name"] for artist in track["artists"]])
            album = track["album"]["name"]
            release_date = track["album"]["release_date"]
            track_id = track['id']

            audio_features_url = f"https://api.spotify.com/v1/audio-features/{track_id}"
            audio_features_response = requests.get(audio_features_url, headers=headers)

            if audio_features_response.status_code == 200:
                audio_features = audio_features_response.json()
                tempo = audio_features.get("tempo", "N/A")
                danceability = audio_features.get("danceability", "N/A")
                energy = audio_features.get("energy", "N/A")
                valence = audio_features.get("valence", "N/A")

                print(
                    f"{artists} - {album} - {release_date} - "
                    f"{tempo:.2f} BPM - {danceability:.2f} - "
                    f"{energy:.2f} - {valence:.2f} - "
                    f"{track['external_urls']['spotify']}"
                )
            else:
                print(f"Erreur lors de la récupération des caractéristiques audio : {audio_features_response.status_code}")
    else:
        print(f"Erreur lors de la recherche de morceaux : {search_response.status_code}")



Entrez un genre musical : rap
Artiste - Album - Date - Tempo - Danceability - Energy - Valence - Lien
---------------------------------------------------------------------
Travis Scott - HIGHEST IN THE ROOM - 2019-10-04 - 76.47 BPM - 0.60 - 0.43 - 0.06 - https://open.spotify.com/track/3eekarcy7kvN4yt5ZFzltW
$uicideboy$ - Antarctica - 2016-01-26 - 105.94 BPM - 0.71 - 0.63 - 0.32 - https://open.spotify.com/track/5UGAXwbA17bUC0K9uquGY2
Drake - No Face - 2024-08-31 - 125.95 BPM - 0.85 - 0.69 - 0.47 - https://open.spotify.com/track/1560osUcXrnov6yuOjXvc1
J. Cole - Cole World: The Sideline Story - 2011-09-27 - 93.00 BPM - 0.83 - 0.43 - 0.22 - https://open.spotify.com/track/2wAJTrFhCnQyNSD3oUgTZO
Post Malone, Luke Combs - F-1 Trillion - 2024-08-15 - 119.99 BPM - 0.61 - 0.91 - 0.49 - https://open.spotify.com/track/6StwwqB84sJeLr7tZDTxEX
